# Project 2 - Build Your Own Custom Chatbot

Decided to use `2023_fashion_trends.csv` dataset for this project because the cutoff of gpt-3.5 is in 2021 thus lacking recent fashion trends. 

In [85]:
# install requirements.txt
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data wrangling

Plan:
- Inspect shape, columns, and null counts
- Clean the `Trends` text into a normalized `text` column
- Preview a few cleaned examples


In [86]:
import os
import time
import traceback
import pandas as pd

# Debug utilities
DEBUG = True

def debug_print(*args, **kwargs):
    if DEBUG:
        print(*args, **kwargs)

pd.set_option("display.max_colwidth", 200)

In [87]:
df = pd.read_csv("data/2023_fashion_trends.csv")
debug_print(f"Loaded dataframe with shape: {df.shape}")
debug_print(f"Columns: {list(df.columns)}")
debug_print("Null counts:", df.isnull().sum().to_dict())
debug_print("First 2 rows:")
debug_print(df.head(2))
df

Loaded dataframe with shape: (82, 3)
Columns: ['URL', 'Trends', 'Source']
Null counts: {'URL': 0, 'Trends': 0, 'Source': 0}
First 2 rows:
                                                    URL  \
0  https://www.refinery29.com/en-us/fashion-trends-2023   
1  https://www.refinery29.com/en-us/fashion-trends-2023   

                                                                                                                                                                                                    Trends  \
0  2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that wou...   
1  2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, ...   

                                                      Source  
0 

,URL,Trends,Source
0,https://www.refinery29.com/en-us/fashion-trends-2023,2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that wou...,7 Fashion Trends That Will Take Over 2023 — Shop Them Now
1,https://www.refinery29.com/en-us/fashion-trends-2023,"2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, ...",7 Fashion Trends That Will Take Over 2023 — Shop Them Now
2,https://www.refinery29.com/en-us/fashion-trends-2023,"2023 Fashion Trend: Sheer Clothing. ""Bare it all"" has been the motto since the end of the lockdown. In 2023, naked dressing makes its way from the red carpet – where celebrities like Cher and Rih...",7 Fashion Trends That Will Take Over 2023 — Shop Them Now
3,https://www.refinery29.com/en-us/fashion-trends-2023,"2023 Fashion Trend: Denim Reimagined. From double-waisted jeans to carpenter jeans, it's been a while since we were this excited about denim trends. It seems like even the most luxe runway designe...",7 Fashion Trends That Will Take Over 2023 — Shop Them Now
4,https://www.refinery29.com/en-us/fashion-trends-2023,2023 Fashion Trend: Shine For The Daytime. The amount of shine on the 2023 runways would make you think we are back in the Roaring '20s. While sequins may feel over as soon as the clock strikes mi...,7 Fashion Trends That Will Take Over 2023 — Shop Them Now
...,...,...,...
77,https://www.whowhatwear.com/spring-summer-2023-fashion-trends/,"If lime green isn't your vibe, rest assured there was another bold hue that practically jumped off the runway and my screen, which was evident from the moment I began this lengthy research process...",Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See
78,https://www.whowhatwear.com/spring-summer-2023-fashion-trends/,"""As someone who can clearly (not fondly) remember their first bra fitting, which resulted in the teary-eyed purchase of what could only be described at the time as 'over-shoulder boulder holders,'...",Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See
79,https://www.whowhatwear.com/spring-summer-2023-fashion-trends/,"""Combine this design shift with the fact that we're moving into a fifth wave of feminism where many predisposed notions and ideologies tied up with previous movements are being flipped on their he...",Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See
80,https://www.whowhatwear.com/spring-summer-2023-fashion-trends/,Thought party season ended at the stroke of midnight on December 31? Think again! The spring/summer runways bear a striking resemblance to all of my favourite coming-of-age movies from the '80s—Si...,Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See


The data has three columns:

- **URL**: the webpage where the trend information is sourced
- **Trends**: a text description of a specific fashion trend
- **Source**: the name/title of the article, blog post, or publication

We will focus on `Trends` and create a cleaned `text` column for prompting.

Tip: debug prints above show shape, columns and null counts to quickly catch data issues.

In [88]:
df["Trends"][0]

'2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessory pieces (shoes, handbags, jewelry).'

In [89]:
df["Trends"][78]

'"As someone who can clearly (not fondly) remember their first bra fitting, which resulted in the teary-eyed purchase of what could only be described at the time as \'over-shoulder boulder holders,\' it\'s surprising that I\'d now find myself backing this brassiere-revealing trend. Or perhaps that\'s the point! Lingerie styles for big (and small) cups have improved vastly over the past 10 years, so why not celebrate it?" says Almassi.'

We will normalize the `Trends` text (remove stray characters, collapse whitespace) and store the result in a new `text` column.

In [90]:
import re

def clean_text(text: str) -> str:
    """
    Normalize a trend snippet:
    - Remove stray backslashes
    - Strip non-standard chars (keep letters, digits, punctuation, whitespace)
    - Collapse repeated whitespace
    """
    if pd.isna(text):
        return ""
    text = str(text).replace("\\", "")
    cleaned = re.sub(r"[^a-zA-Z0-9\s.,'\"!?;:()-]", "", text)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

# Apply cleaning with lightweight debugging
sample_before = df["Trends"].astype(str).head(3).tolist()
df["text"] = df["Trends"].apply(clean_text)
sample_after = df["text"].head(3).tolist()
debug_print("Sample clean_text before/after (first 3):")
for i, (b, a) in enumerate(zip(sample_before, sample_after), 1):
    debug_print(f"{i}. before: {b[:120]}...")
    debug_print(f"   after : {a[:120]}...")

Sample clean_text before/after (first 3):
1. before: 2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burc...
   after : 2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burc...
2. before: 2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant....
   after : 2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant....
3. before: 2023 Fashion Trend: Sheer Clothing. "Bare it all" has been the motto since the end of the lockdown. In 2023,  naked dres...
   after : 2023 Fashion Trend: Sheer Clothing. "Bare it all" has been the motto since the end of the lockdown. In 2023, naked dress...


In [91]:
df

,URL,Trends,Source,text
0,https://www.refinery29.com/en-us/fashion-trends-2023,2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that wou...,7 Fashion Trends That Will Take Over 2023 — Shop Them Now,2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that wou...
1,https://www.refinery29.com/en-us/fashion-trends-2023,"2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, ...",7 Fashion Trends That Will Take Over 2023 — Shop Them Now,"2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, ..."
2,https://www.refinery29.com/en-us/fashion-trends-2023,"2023 Fashion Trend: Sheer Clothing. ""Bare it all"" has been the motto since the end of the lockdown. In 2023, naked dressing makes its way from the red carpet – where celebrities like Cher and Rih...",7 Fashion Trends That Will Take Over 2023 — Shop Them Now,"2023 Fashion Trend: Sheer Clothing. ""Bare it all"" has been the motto since the end of the lockdown. In 2023, naked dressing makes its way from the red carpet where celebrities like Cher and Rihann..."
3,https://www.refinery29.com/en-us/fashion-trends-2023,"2023 Fashion Trend: Denim Reimagined. From double-waisted jeans to carpenter jeans, it's been a while since we were this excited about denim trends. It seems like even the most luxe runway designe...",7 Fashion Trends That Will Take Over 2023 — Shop Them Now,"2023 Fashion Trend: Denim Reimagined. From double-waisted jeans to carpenter jeans, it's been a while since we were this excited about denim trends. It seems like even the most luxe runway designe..."
4,https://www.refinery29.com/en-us/fashion-trends-2023,2023 Fashion Trend: Shine For The Daytime. The amount of shine on the 2023 runways would make you think we are back in the Roaring '20s. While sequins may feel over as soon as the clock strikes mi...,7 Fashion Trends That Will Take Over 2023 — Shop Them Now,2023 Fashion Trend: Shine For The Daytime. The amount of shine on the 2023 runways would make you think we are back in the Roaring '20s. While sequins may feel over as soon as the clock strikes mi...
...,...,...,...,...
77,https://www.whowhatwear.com/spring-summer-2023-fashion-trends/,"If lime green isn't your vibe, rest assured there was another bold hue that practically jumped off the runway and my screen, which was evident from the moment I began this lengthy research process...",Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See,"If lime green isn't your vibe, rest assured there was another bold hue that practically jumped off the runway and my screen, which was evident from the moment I began this lengthy research process..."
78,https://www.whowhatwear.com/spring-summer-2023-fashion-trends/,"""As someone who can clearly (not fondly) remember their first bra fitting, which resulted in the teary-eyed purchase of what could only be described at the time as 'over-shoulder boulder holders,'...",Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See,"""As someone who can clearly (not fondly) remember their first bra fitting, which resulted in the teary-eyed purchase of what could only be described at the time as 'over-shoulder boulder holders,'..."
79,https://www.whowhatwear.com/spring-summer-2023-fashion-trends/,"""Combine this design shift with the fact that we're moving into a fifth wave of feminism where many predisposed notions and ideologies tied up with previous movements are being flipped on their he...",Spring/Summer

In [92]:
print(df["text"][78])

"As someone who can clearly (not fondly) remember their first bra fitting, which resulted in the teary-eyed purchase of what could only be described at the time as 'over-shoulder boulder holders,' it's surprising that I'd now find myself backing this brassiere-revealing trend. Or perhaps that's the point! Lingerie styles for big (and small) cups have improved vastly over the past 10 years, so why not celebrate it?" says Almassi.


## Custom query completion

We build one context string from the `text` column (one line per row) and pass it to the chat model. We will compare answers with and without the context. Debug prints will show prompt sizes and request latency.

In [93]:
from openai import OpenAI

BASE_URL = os.getenv("OPENAI_BASE_URL", "https://openai.vocareum.com/v1")
API_KEY = os.getenv("OPENAI_API_KEY", "").strip()
if not API_KEY:
    raise ValueError("OPENAI_API_KEY is not set. Please set it in your environment.")

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

def _obfuscate(key: str, show: int = 4) -> str:
    return (key[:show] + "...") if key else ""

debug_print(f"Using base URL: {BASE_URL}")
debug_print(f"API key starts with: {_obfuscate(API_KEY)}")

Using base URL: https://openai.vocareum.com/v1
API key starts with: voc-...


In [94]:
# Build a single context string from the dataset
context = "\n\n".join(df["text"].astype(str).tolist())
debug_print(f"Context length (chars): {len(context):,}")
approx_tokens = len(context.split())
debug_print(f"Approx tokens (whitespace split): {approx_tokens:,}")
context[:500]  # preview the start of the context string

Context length (chars): 35,552
Approx tokens (whitespace split): 5,870


"2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessory pieces (shoes, handbags, jewelry).\n\n2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For "

### Custom Query 1

In [95]:
# Custom query 1
custom_query_1 = "Which 2023 fashion trend emphasizes sustainability or eco-friendly practices?"

prompt_1 = f"Answer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {custom_query_1}\nAnswer:"

start = time.time()
try:
    response_1 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_1}],
        max_tokens=150
    )
    latency_ms = (time.time() - start) * 1000
    answer_1 = response_1.choices[0].message.content.strip()
    debug_print(f"[Q1] Prompt chars: {len(prompt_1):,} | Latency: {latency_ms:.0f} ms")
    print(f"Custom Query: {custom_query_1}")
    print(f"Answer: {answer_1}")
except Exception as e:
    print("Request failed:", e)
    traceback.print_exc()

[Q1] Prompt chars: 35,707 | Latency: 2603 ms
Custom Query: Which 2023 fashion trend emphasizes sustainability or eco-friendly practices?
Answer: The motion towards minimalist and simpler designs with a focus on tailored silhouettes and clean lines emphasizes sustainability or eco-friendly practices in the 2023 fashion trends. Brands like Bottega Veneta, Valentino, The Row, and Ferragamo are embracing simplicity and everyday dressing, which can indicate a move towards more sustainable and responsible fashion choices.


### Custom Query 2

In [96]:
# Custom query 2
custom_query_2 = "Summarize three notable 2023 fashion trends."

prompt_2 = f"Answer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {custom_query_2}\nAnswer:"

start = time.time()
try:
    response_2 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_2}],
        max_tokens=150
    )
    latency_ms = (time.time() - start) * 1000
    answer_2 = response_2.choices[0].message.content.strip()
    debug_print(f"[Q2] Prompt chars: {len(prompt_2):,} | Latency: {latency_ms:.0f} ms")
    print(f"Custom Query: {custom_query_2}")
    print(f"Answer: {answer_2}")
except Exception as e:
    print("Request failed:", e)
    traceback.print_exc()

[Q2] Prompt chars: 35,674 | Latency: 2974 ms
Custom Query: Summarize three notable 2023 fashion trends.
Answer: 1. Sheer Clothing: The trend of sheer clothing has made its way from the red carpet to street style, with designers experimenting with barely-there fabrics and see-through designs.

2. Denim Reimagined: Denim trends have taken a new turn in 2023, with tailored and interesting pocket placements, along with the use of elevated fabrics like silk and organza in denim pieces.

3. Bold Color Bottoms: Bright and bold color bottoms are a key trend for 2023, with shades like lime green and saffron making a statement in head-to-toe looks and accent pieces.


## Custom performance demonstration

We compare answers from the base model (no context) and the model with our CSV-derived context. This highlights how providing targeted context can improve relevance. Timing and prompt size are printed for quick debugging.

### Question 1

In [97]:
# No context query
question1 = "Which 2023 fashion trend focuses on sustainability?"

start = time.time()
try:
    response_basic1 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": question1}],
        max_tokens=150
    )
    latency_ms = (time.time() - start) * 1000
    answer_basic1 = response_basic1.choices[0].message.content.strip()
    debug_print(f"[Q1-Base] Prompt chars: {len(question1):,} | Latency: {latency_ms:.0f} ms")
    print(f"Basic Model Answer (Q1): {answer_basic1}")
except Exception as e:
    print("Request failed:", e)
    traceback.print_exc()

[Q1-Base] Prompt chars: 51 | Latency: 1723 ms
Basic Model Answer (Q1): One 2023 fashion trend that focuses on sustainability is the rise of eco-friendly materials and practices in the fashion industry. This includes a move towards using more sustainable fabrics like organic cotton, bamboo, and recycled materials, as well as implementing more environmentally friendly production processes such as waterless dyeing techniques and reducing waste in the supply chain. Additionally, there has been a push towards circular fashion, where garments are designed to be easily recycled or upcycled at the end of their life cycle. Overall, the trend towards sustainability in fashion in 2023 is all about creating a more ethical and environmentally conscious industry.


In [98]:
# With context query
prompt_custom1 = f"Answer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {question1}\nAnswer:"

start = time.time()
try:
    response_custom1 = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_custom1}],
        max_tokens=150
    )
    latency_ms = (time.time() - start) * 1000
    answer_custom1 = response_custom1.choices[0].message.content.strip()
    debug_print(f"[Q1-Context] Prompt chars: {len(prompt_custom1):,} | Latency: {latency_ms:.0f} ms")
    print(f"Custom Query Answer (Q1): {answer_custom1}")
except Exception as e:
    print("Request failed:", e)
    traceback.print_exc()

[Q1-Context] Prompt chars: 35,681 | Latency: 1583 ms
Custom Query Answer (Q1): Some of the sustainable fashion trends for 2023 include 'Elevated Basics, ' 'Detailed Denim, ' and 'Tailored Look.'
